# Object Following - Live Demo

In this notebook we'll show how you can follow an object with JetBot!  We'll use a pre-trained neural network
that was trained on the [COCO dataset](http://cocodataset.org) to detect 90 different common objects.  These include

* Person (index 0)
* Cup (index 47)

and many others (you can check [this file](https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt) for a full list of class indices).  The model is sourced from the [TensorFlow object detection API](https://github.com/tensorflow/models/tree/master/research/object_detection)
which provides utilities for training object detectors for custom tasks also!  Once the model is trained, we optimize it using NVIDIA TensorRT on the Jetson Nano.

This makes the network very fast, capable of real-time execution on Jetson Nano!  We won't run through all of the training and optimization steps in this notebook though.

Anyways, let's get started.  First, we'll want to import the ``Object_Follower`` class which takes our pre-trained SSD engine or yolo engine.

In [ ]:
from IPython.display import display
import ipywidgets.widgets as widgets
from IPython.display import clear_output

from ipywidgets.widgets import HBox, Layout
import traitlets


In [ ]:
from jetbot.utils import model_selection as ms

trt_ms = ms.trt_model_selection()

model_function_widget = widgets.Select(options=trt_ms.model_function_list, value="object detection",
                                       description='Model Function:')
traitlets.dlink((model_function_widget, 'value'), (trt_ms, 'model_function'))
traitlets.dlink((trt_ms, 'model_function_list'), (model_function_widget, 'options'))

model_type_widget = widgets.Select(options=trt_ms.model_type_list, value="SSD", description='Model Type:')
traitlets.dlink((model_type_widget, 'value'), (trt_ms, 'model_type'))
traitlets.dlink((trt_ms, 'model_type_list'), (model_type_widget, 'options'))

model_path_widget = widgets.Select(options=trt_ms.model_path_list, description='Model Path:',
                                   layout=Layout(width='50%'))
traitlets.dlink((model_path_widget, 'value'), (trt_ms, 'model_path'))
traitlets.dlink((trt_ms, 'model_path_list'), (model_path_widget, 'options'))

display(HBox([model_function_widget, model_type_widget, model_path_widget]))

In [ ]:
type_follower_model = trt_ms.model_type
follower_model = trt_ms.selected_model_path
print('path of object detector model: %s' % follower_model)
trt_ms.unobserve_all()
trt_ms = None

In [ ]:
from jetbot import ObjectFollower

avoider_model='../collision_avoidance/best_model.pth'

OF = ObjectFollower(follower_model=follower_model, avoider_model=avoider_model, type_follower_model=type_follower_model, conf_th=0.3)

In [ ]:
out = widgets.Output()

# image_widget = widgets.Image(format='jpeg', width=OF.img_width, height=OF.img_height)
image_widget = widgets.Image(format='jpeg', width=350, height=350)

# display(image_widget)
traitlets.dlink((OF, 'cap_image'), (image_widget, 'value'))

# display buttons
button_layout = widgets.Layout(width='100px', height='40px', align_self='center')
stop_button = widgets.Button(description='Stop', button_style='danger', tooltip='Click to stop running', icon='fa-stop', layout=button_layout)
start_button = widgets.Button(description='Start', tooltip='Click to start running', layout=button_layout)
button_box = widgets.HBox([start_button, stop_button], layout=widgets.Layout(align_self='center'))


In [ ]:
blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
label_widget = widgets.IntText(value=1, description='tracked label')  # target to be tracked
label_text_widget = widgets.Text(value='', description='label name')  # target name to be tracked
speed_widget = widgets.FloatSlider(value=0.18, min=0.05, max=0.5, step=0.001, description='speed', readout_format='.3f')
turn_gain_widget = widgets.FloatSlider(value=0.25, min=0.05, max=0.5, step=0.001, description='turn gain', readout_format='.3f')
steering_bias_widget = widgets.FloatSlider(value=0.02, min=-0.1, max=0.1, step=0.001, description='steering bias', readout_format='.3f')

traitlets.dlink((OF, 'blocked'), (blocked_widget, 'value'))
traitlets.dlink((label_widget, 'value'), (OF, 'label'))
traitlets.dlink((OF, 'label_text'), (label_text_widget, 'value'))
traitlets.dlink((turn_gain_widget, 'value'), (OF, 'turn_gain'))
traitlets.dlink((speed_widget, 'value'), (OF, 'speed'))
traitlets.dlink((steering_bias_widget, 'value'), (OF, 'steering_bias'))

In [ ]:
def start(change):
    OF.start_run(change)

def stop(change):
    OF.stop_run(change)
    %reset -f

In [ ]:
display(widgets.VBox([
    widgets.HBox([image_widget, blocked_widget]),
    widgets.HBox([label_widget, label_text_widget]),
    speed_widget,
    turn_gain_widget,
    steering_bias_widget,
    button_box
]))

start_button.on_click(start)
stop_button.on_click(stop)